<a href="https://colab.research.google.com/github/Eratofee/WaterMusicGeneration/blob/main/WaterMusicGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install mido pretty_midi
!pip install midiutil
!apt-get -y -qq update
!apt-get -y -qq install fluidsynth fluid-soundfont-gm

In [ ]:
import numpy as np
import pandas as pd
import base64, io, os
import matplotlib.pyplot as plt
from IPython.display import Audio
from IPython.display import HTML, display
import pretty_midi

In [ ]:
file_path = "DataSet_20230217-20231231.csv"
df = pd.read_csv(file_path)

# Datum in echtes Zeitformat umwandeln
df["date"] = pd.to_datetime(df["date"], dayfirst=True)

df.head()
print(df.columns)
# df

In [ ]:
# ---------- instrument config ----------
# GM program numbers are 1..128
INSTRUMENTS = [
    {"station":"P343",  "instrument_name":"Violin",       "gm_program":41, "low":"G3",  "high":"E7"},
    {"station":"P33",   "instrument_name":"Clarinet",     "gm_program":72, "low":"E3",  "high":"C7"},
    {"station":"P501",  "instrument_name":"Cello",        "gm_program":43, "low":"C2",  "high":"A5"},  # praxisnah
    {"station":"P37",   "instrument_name":"Trombone",     "gm_program":58, "low":"E2",  "high":"F5"},
    {"station":"P349",  "instrument_name":"Contrabass",   "gm_program":44, "low":"E1",  "high":"C5"},
    {"station":"P38",   "instrument_name":"Bassoon",      "gm_program":71, "low":"Bb1", "high":"Eb5"},
    {"station":"P9",    "instrument_name":"Church Organ", "gm_program":20, "low":"C2",  "high":"C7"},
    {"station":"P9",    "instrument_name":"Trumpet",      "gm_program":57, "low":"F#3", "high":"E6"},
    {"station":"P618",  "instrument_name":"Oboe",         "gm_program":69, "low":"Bb3", "high":"A6"},
    {"station":"GN-LRu","instrument_name":"Synth Pad 1",  "gm_program":89, "low":"C2",  "high":"C7"},
]

In [ ]:
# If "date" is a column, use it as x-axis
if "date" in df.columns:
    x = df["date"]
    y_df = df.drop(columns=["date"])
else:
    x = df.index
    y_df = df

plt.figure(figsize=(14, 6))

# Alternative to y_df:
y_df_smooth = y_df.rolling(100, min_periods=1).mean()

for col in y_df_smooth.columns:
    plt.plot(x, y_df_smooth[col], label=col, alpha=0.6)

plt.xlabel("Date")
plt.ylabel("Water Level")
plt.title("Overview of All Stations")
plt.grid(True)


# Put legend outside for readability
plt.legend(loc="upper left", bbox_to_anchor=(1.02, 1.0))
plt.tight_layout()
plt.show()


In [ ]:
# --- 0) Ensure df is in the expected shape: time column or time index ---
df2 = df.copy()

# If time is in a column called "date", use it; otherwise use index as time
if "date" in df2.columns:
    df2["date"] = pd.to_datetime(df2["date"], dayfirst=True, errors="coerce")
    df2 = df2.sort_values("date")
    time_col = "date"
else:
    # assume index is datetime-like
    df2 = df2.copy()
    df2.index = pd.to_datetime(df2.index, errors="coerce")
    df2 = df2.sort_index()
    df2 = df2.reset_index().rename(columns={"index": "date"})
    time_col = "date"

# --- 1) Station -> Instrument mapping (GM program numbers 1..128) ---
# GM reference (common):
# Violin 41, Clarinet 72, Cello 43, Trombone 58, Contrabass 44,
# Bassoon 71, Church Organ 20, Trumpet 57, Oboe 69, Synth Pad 1 89
mapping = pd.DataFrame([
    {"station": "P343",  "instrument_name": "Violin",        "midi_program_gm_1_128": 41},
    {"station": "P33",   "instrument_name": "Clarinet",      "midi_program_gm_1_128": 72},
    {"station": "P501",  "instrument_name": "Cello",         "midi_program_gm_1_128": 43},
    {"station": "P37",   "instrument_name": "Trombone",      "midi_program_gm_1_128": 58},
    {"station": "P349",  "instrument_name": "Contrabass",    "midi_program_gm_1_128": 44},
    {"station": "P38",   "instrument_name": "Bassoon",       "midi_program_gm_1_128": 71},
    {"station": "P9",    "instrument_name": "Church Organ",  "midi_program_gm_1_128": 20},
    {"station": "P9",    "instrument_name": "Trumpet",       "midi_program_gm_1_128": 57},  # duplicate station on purpose
    {"station": "P618",  "instrument_name": "Oboe",          "midi_program_gm_1_128": 69},
    {"station": "GN-LRu","instrument_name": "Synth Pad 1",   "midi_program_gm_1_128": 89},
])

# --- 2) Build long/tidy df: time, station, value ---
# Keep only stations that appear in mapping
stations = mapping["station"].unique().tolist()
available = [c for c in stations if c in df2.columns]

long_df = df2[[time_col] + available].melt(
    id_vars=[time_col],
    var_name="station",
    value_name="value"
).dropna(subset=["value"])

# --- 3) Attach instrument info (duplicates on P9 will duplicate rows intentionally) ---
assigned_df = long_df.merge(mapping, on="station", how="left")

# --- 4) Rename time column to exactly 'time' as requested, and order columns ---
assigned_df = assigned_df.rename(columns={time_col: "time"})
assigned_df = assigned_df[["time", "station", "value", "instrument_name", "midi_program_gm_1_128"]]

assigned_df.head(10), assigned_df.shape


In [ ]:
# ---------- helpers: note name <-> MIDI ----------
_NOTE2SEMI = {"C":0,"C#":1,"Db":1,"D":2,"D#":3,"Eb":3,"E":4,"F":5,"F#":6,"Gb":6,"G":7,"G#":8,"Ab":8,"A":9,"A#":10,"Bb":10,"B":11}
def note_to_midi(note: str) -> int:
    # e.g. "G3", "Bb1", "F#5"
    note = note.strip()
    if len(note) < 2:
        raise ValueError(f"Bad note: {note}")
    # split pitch class and octave
    if note[1] in ["b", "#"]:
        pc = note[:2]
        octv = int(note[2:])
    else:
        pc = note[:1]
        octv = int(note[1:])
    return 12*(octv + 1) + _NOTE2SEMI[pc]  # MIDI: C4=60

def build_dorian_scale(root_midi: int) -> np.ndarray:
    # Dorian intervals: 0,2,3,5,7,9,10 (relative to root)
    return np.array([root_midi + i for i in [0,2,3,5,7,9,10]], dtype=int)

def build_scale_in_range(root_note: str = "D3", low: int = 48, high: int = 72) -> np.ndarray:
    # Generate D-dorian pitches between [low, high] inclusive.
    root = note_to_midi(root_note)
    pitches = []
    # sweep octaves around the target range
    for o in range(-6, 10):
        pitches.extend(build_dorian_scale(root + 12*o).tolist())
    pitches = np.array(sorted(set(pitches)))
    pitches = pitches[(pitches >= low) & (pitches <= high)]
    if len(pitches) < 2:
        raise ValueError(f"Scale too small in range low={low}, high={high}.")
    return pitches

def quantize_values_to_scale(values: np.ndarray, scale: np.ndarray) -> np.ndarray:
    """Map min(values)->scale[0], max(values)->scale[-1], linearly then quantize to nearest scale index."""
    v = np.asarray(values, dtype=float)
    vmin = np.nanmin(v)
    vmax = np.nanmax(v)
    if not np.isfinite(vmin) or not np.isfinite(vmax) or abs(vmax - vmin) < 1e-12:
        # constant or invalid -> middle pitch
        return np.full(len(v), int(scale[len(scale)//2]), dtype=int)
    t = (v - vmin) / (vmax - vmin)
    idx = np.rint(t * (len(scale)-1)).astype(int)
    idx = np.clip(idx, 0, len(scale)-1)
    return scale[idx].astype(int)


def map_data_to_midi(df: pd.DataFrame,
                     instruments=INSTRUMENTS,
                     time_col: str = "date",
                     scale_root: str = "D3") -> pd.DataFrame:
    """
    Returns tidy DF with columns:
    time, station, instrument_name, gm_program, value, midi_pitch
    """
    d = df.copy()
    if time_col in d.columns:
        d[time_col] = pd.to_datetime(d[time_col], dayfirst=True, errors="coerce")
        d = d.sort_values(time_col)
    else:
        # assume index is time
        d = d.copy()
        d.index = pd.to_datetime(d.index, errors="coerce")
        d = d.sort_index()
        d = d.reset_index().rename(columns={"index": time_col})

    out_rows = []
    for inst in instruments:
        st = inst["station"]
        if st not in d.columns:
            continue

        series = d[[time_col, st]].rename(columns={st:"value"}).copy()
        series["value"] = pd.to_numeric(series["value"], errors="coerce")
        # fill missing values (needed to define min/max robustly across the piece)
        series["value"] = series["value"].interpolate(limit_direction="both")
        series = series.dropna(subset=["value"])

        low_m = note_to_midi(inst["low"])
        high_m = note_to_midi(inst["high"])
        scale = build_scale_in_range(root_note=scale_root, low=low_m, high=high_m)

        midi_pitch = quantize_values_to_scale(series["value"].to_numpy(), scale)

        tmp = pd.DataFrame({
            "time": series[time_col].to_numpy(),
            "station": st,
            "instrument_name": inst["instrument_name"],
            "gm_program": inst["gm_program"],
            "value": series["value"].to_numpy(),
            "midi_pitch": midi_pitch
        })
        out_rows.append(tmp)

    mapped = pd.concat(out_rows, ignore_index=True)
    # track label for notation tools
    mapped["track_name"] = mapped["instrument_name"] + " (" + mapped["station"] + ")"
    return mapped

# ---------- rhythm + MIDI rendering ----------
def render_rhythmic_midi(mapped_df: pd.DataFrame,
                         bars: int = 64,
                         tempo_bpm: int = 80,
                         time_signature=(4,4),
                         quarter_weight: float = 0.55,
                         seed: int = 7,
                         ticks_per_beat: int = 480,
                         outfile: str = "water_piece_64bars.mid"):
    """
    Creates a GM MIDI file with named tracks. Rhythm grid is 1/8 notes, durations allowed:
    eighth (0.5), quarter (1), half (2), whole (4) beats.
    Highest probability for quarter note by default.
    """
    # lazy import / install hint
    try:
        import mido
        from mido import Message, MetaMessage
    except Exception as e:
        raise ImportError("Please install mido first (in Colab: !pip -q install mido).") from e

    rng = np.random.default_rng(seed)
    beats_per_bar = time_signature[0] * (4 / time_signature[1])
    total_beats = int(round(bars * beats_per_bar))

    # Base grid: eighth notes => 2 steps per beat
    steps_per_beat = 2
    total_steps = total_beats * steps_per_beat  # each step = 1/8
    step_len_beats = 1 / steps_per_beat

    # Allowed durations in steps (1=8th, 2=quarter, 4=half, 8=whole)
    dur_steps = np.array([1, 2, 4, 8], dtype=int)

    # Weights: quarter highest by requirement
    # You can tweak these weights if you want “more long notes”.
    # quarter_weight controls the quarter note dominance.
    # Distribute remaining probability over other values:
    rem = 1.0 - quarter_weight
    weights = np.array([rem*0.25, quarter_weight, rem*0.40, rem*0.35], dtype=float)
    weights = weights / weights.sum()

    # Resample each track's pitch stream to exactly total_steps points
    tracks = []
    for track_name, g in mapped_df.groupby("track_name", sort=False):
        g = g.sort_values("time")
        pitches = g["midi_pitch"].to_numpy().astype(int)

        # If data very long: compress via index-space interpolation (keeps global contour)
        if len(pitches) == 0:
            continue
        x_old = np.linspace(0, 1, num=len(pitches))
        x_new = np.linspace(0, 1, num=total_steps)
        pitches_rs = np.interp(x_new, x_old, pitches).round().astype(int)

        tracks.append({
            "track_name": track_name,
            "gm_program": int(g["gm_program"].iloc[0]),
            "pitches_steps": pitches_rs
        })

    # Helper: turn stepwise pitches into note events with allowed durations
    def steps_to_events(pitches_steps: np.ndarray):
        # run-length encode identical pitches
        events = []
        i = 0
        while i < len(pitches_steps):
            p = int(pitches_steps[i])
            j = i + 1
            while j < len(pitches_steps) and int(pitches_steps[j]) == p:
                j += 1
            run_len = j - i  # in steps

            # split this run into allowed dur_steps with probabilistic preference
            remaining = run_len
            while remaining > 0:
                possible = dur_steps[dur_steps <= remaining]
                w = weights[:len(possible)].copy()
                w = w / w.sum()
                d = int(rng.choice(possible, p=w))
                events.append((p, d))
                remaining -= d

            i = j
        return events

    # Build MIDI
    mid = mido.MidiFile(type=1, ticks_per_beat=ticks_per_beat)

    # Global meta track
    meta = mido.MidiTrack()
    mid.tracks.append(meta)
    meta.append(MetaMessage("track_name", name="Score", time=0))
    meta.append(MetaMessage("time_signature", numerator=time_signature[0], denominator=time_signature[1], time=0))
    meta.append(MetaMessage("set_tempo", tempo=mido.bpm2tempo(tempo_bpm), time=0))

    # One track per instrument
    for tr in tracks:
        t = mido.MidiTrack()
        mid.tracks.append(t)
        t.append(MetaMessage("track_name", name=tr["track_name"], time=0))
        # GM program change: mido uses 0..127 internally
        t.append(Message("program_change", program=tr["gm_program"]-1, time=0, channel=0))

        events = steps_to_events(tr["pitches_steps"])

        # Write note events with delta-times in ticks
        # Each step is an eighth note => ticks = ticks_per_beat/2
        step_ticks = int(ticks_per_beat / steps_per_beat)
        velocity = 90

        for pitch, d_steps in events:
            dur_ticks = d_steps * step_ticks
            t.append(Message("note_on", note=int(pitch), velocity=velocity, time=0, channel=0))
            t.append(Message("note_off", note=int(pitch), velocity=0, time=dur_ticks, channel=0))

    mid.save(outfile)
    return outfile


In [ ]:
# --------- Example usage ----------
mapped = map_data_to_midi(df, scale_root="D3")
midi_path = render_rhythmic_midi(mapped, bars=64, tempo_bpm=80, time_signature=(4,4))
print("Wrote:", midi_path)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def midi_to_partiture_image(
    midi_path: str,
    bars: int = 64,
    time_signature=(4, 4),
    outfile_png: str | None = None,
    figsize=(20, 6),
    min_note_len_beats=0.125,   # discard micro-blips
    show_pitch_range=False      # if True, shows pitch min/max per track on the left
):
    """
    Create a one-line (all bars in one row on x-axis) 'partiture image' from a MIDI file.
    Each MIDI track becomes one staff-row (voice). Notes are drawn as horizontal bars.

    X-axis: beats (optionally labeled by bar numbers)
    Y-axis: voices (tracks)
    """

    # --- Import mido lazily (clear error message if missing) ---
    try:
        import mido
    except Exception as e:
        raise ImportError("mido is required. Install via: pip install mido (Colab: !pip -q install mido)") from e

    mid = mido.MidiFile(midi_path)
    tpq = mid.ticks_per_beat

    beats_per_bar = time_signature[0] * (4 / time_signature[1])
    total_beats = bars * beats_per_bar

    # --- Helper: extract track name ---
    def get_track_name(track, default):
        for msg in track:
            if msg.type == "track_name":
                return msg.name
        return default

    # --- Parse notes per track into (start_beat, duration_beats, pitch) ---
    tracks_notes = []
    for ti, track in enumerate(mid.tracks):
        name = get_track_name(track, f"Track {ti}")

        # We typically skip the global meta track ("Score") if it has no notes
        abs_ticks = 0
        on = {}  # (channel, note) -> start_tick
        notes = []

        for msg in track:
            abs_ticks += msg.time
            if msg.type == "note_on" and msg.velocity > 0:
                on[(getattr(msg, "channel", 0), msg.note)] = abs_ticks
            elif (msg.type == "note_off") or (msg.type == "note_on" and msg.velocity == 0):
                key = (getattr(msg, "channel", 0), msg.note)
                if key in on:
                    start = on.pop(key)
                    dur = abs_ticks - start
                    start_beat = start / tpq
                    dur_beats = dur / tpq
                    if dur_beats >= min_note_len_beats:
                        notes.append((start_beat, dur_beats, int(msg.note)))

        if len(notes) > 0:
            tracks_notes.append({"name": name, "notes": notes})

    if not tracks_notes:
        raise ValueError("No note events found in MIDI (or all were filtered out).")

    # --- Build plot ---
    n = len(tracks_notes)
    fig, ax = plt.subplots(figsize=figsize)

    # Y layout: one band per track
    row_height = 0.7
    y_positions = np.arange(n)[::-1]  # top to bottom
    y_map = {i: y_positions[i] for i in range(n)}

    # For each track, compress pitch to a thin band? -> We keep pitch-variation *within the row* by vertical offset.
    # But you asked: "one row per voice", so pitch should not create extra rows.
    # Solution: draw pitch as slight vertical offset within the same row band.
    # Normalize pitch per track to [0..row_height] inside that row.
    for i, tr in enumerate(tracks_notes):
        notes = tr["notes"]
        pitches = np.array([p for (_, _, p) in notes], dtype=float)
        pmin, pmax = pitches.min(), pitches.max()
        if pmax - pmin < 1e-9:
            norm = lambda p: 0.35  # constant mid
        else:
            norm = lambda p: 0.1 + 0.6 * ((p - pmin) / (pmax - pmin))

        y0 = y_map[i]

        for (start_b, dur_b, pitch) in notes:
            # Clip to desired window (0..total_beats)
            if start_b > total_beats:
                continue
            end_b = min(start_b + dur_b, total_beats)
            dur_draw = max(0.0, end_b - start_b)
            if dur_draw <= 0:
                continue

            y = y0 + norm(pitch) * row_height
            ax.broken_barh([(start_b, dur_draw)], (y, 0.12), linewidth=0)

        # Track label on left
        label = tr["name"]
        if show_pitch_range:
            label += f"  [{int(pmin)}–{int(pmax)}]"
        ax.text(-0.5, y0 + 0.35, label, va="center", ha="right")

    # X axis: bar grid
    bar_ticks = np.arange(0, total_beats + 1e-9, beats_per_bar)
    for bt in bar_ticks:
        ax.axvline(bt, linewidth=0.5, alpha=0.3)

    ax.set_xlim(0, total_beats)
    ax.set_ylim(-0.5, n - 0.5 + row_height)
    ax.set_yticks([])

    ax.set_xlabel("Time (beats) — all bars in one line")
    ax.set_title("MIDI Note Partiture (one row per voice, all bars in one line)")

    # Optional: label every 4 bars to keep it readable
    label_every = 4
    xt = bar_ticks[::label_every]
    ax.set_xticks(xt)
    ax.set_xticklabels([f"{int(b/ beats_per_bar) + 1}" for b in xt])  # bar numbers starting at 1
    ax.set_xlabel(f"Bar number (every {label_every} bars)")

    ax.grid(False)
    plt.tight_layout()

    if outfile_png:
        fig.savefig(outfile_png, dpi=200, bbox_inches="tight")
    return fig, ax


In [ ]:
fig, ax = midi_to_partiture_image(
    "water_piece_64bars.mid",
    bars=64,
    time_signature=(4,4),
    outfile_png="partiture.png",
    figsize=(24, 6)
)


In [ ]:
import os, io, base64, subprocess, tempfile
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML, display

def _find_soundfont():
    candidates = [
        "/usr/share/sounds/sf2/FluidR3_GM.sf2",
        "/usr/share/sounds/sf2/FluidR3_GS.sf2",
    ]
    for p in candidates:
        if os.path.exists(p):
            return p
    raise FileNotFoundError("GM SoundFont not found. Install: apt-get install fluid-soundfont-gm")

def _fig_to_base64_png(fig, dpi=160):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi, bbox_inches="tight")
    plt.close(fig)
    buf.seek(0)
    return base64.b64encode(buf.read()).decode("ascii")

def _file_to_base64(path):
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode("ascii")

def midi_to_wav_bytes_via_cli(midi_path, sr=44100):
    sf2 = _find_soundfont()
    with tempfile.TemporaryDirectory() as td:
        wav_path = os.path.join(td, "out.wav")
        # fluidsynth -ni <soundfont> <midi> -F <wav> -r <sr>
        cmd = ["fluidsynth", "-ni", sf2, midi_path, "-F", wav_path, "-r", str(sr)]
        subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        with open(wav_path, "rb") as f:
            wav_bytes = f.read()
    return wav_bytes

def build_combined_figure(mapped_df, bars=64, time_signature=(4,4),
                          top_tracks=("Violin (P343)", "Cello (P501)"),
                          second_tracks=("Clarinet (P33)", "Trombone (P37)"),
                          figsize=(20,8)):
    beats_per_bar = time_signature[0] * (4 / time_signature[1])
    total_beats = bars * beats_per_bar
    beats_grid = np.linspace(0, total_beats, num=4000)

    def track_curve(track_name, col="value"):
        g = mapped_df[mapped_df["track_name"] == track_name].sort_values("time")
        if len(g) < 2:
            return np.full_like(beats_grid, np.nan, dtype=float)
        y = g[col].to_numpy(dtype=float)
        x_old = np.linspace(0, total_beats, num=len(y))
        return np.interp(beats_grid, x_old, y)

    fig = plt.figure(figsize=figsize)
    gs = fig.add_gridspec(3, 1, height_ratios=[1, 1, 1.4], hspace=0.15)
    ax1 = fig.add_subplot(gs[0,0])
    ax2 = fig.add_subplot(gs[1,0], sharex=ax1)
    ax3 = fig.add_subplot(gs[2,0], sharex=ax1)

    # Top graph
    for tname in top_tracks:
        ax1.plot(beats_grid, track_curve(tname, "value"), label=tname, alpha=0.9)
    ax1.set_ylabel("Value"); ax1.grid(True, alpha=0.25); ax1.legend(loc="upper right")

    # Second graph
    for tname in second_tracks:
        ax2.plot(beats_grid, track_curve(tname, "value"), label=tname, alpha=0.9)
    ax2.set_ylabel("Value"); ax2.grid(True, alpha=0.25); ax2.legend(loc="upper right")

    # Partiture (one row per track)
    tracks = list(mapped_df["track_name"].dropna().unique())
    row_height = 0.8
    y_positions = np.arange(len(tracks))[::-1]
    steps_per_beat = 2  # eighth grid
    total_steps = int(total_beats * steps_per_beat)
    step_len = 1.0 / steps_per_beat

    for i, tname in enumerate(tracks):
        g = mapped_df[mapped_df["track_name"] == tname].sort_values("time")
        if len(g) == 0:
            continue
        pitches = g["midi_pitch"].to_numpy(dtype=float)
        x_old = np.linspace(0, 1, num=len(pitches))
        x_new = np.linspace(0, 1, num=total_steps)
        p_steps = np.interp(x_new, x_old, pitches).round().astype(int)

        y0 = y_positions[i]
        j = 0
        while j < len(p_steps):
            p = p_steps[j]
            k = j + 1
            while k < len(p_steps) and p_steps[k] == p:
                k += 1
            ax3.broken_barh([(j*step_len, (k-j)*step_len)], (y0, 0.18), linewidth=0)
            j = k

        ax3.text(-0.8, y0 + 0.08, tname, va="center", ha="right")

    bar_ticks = np.arange(0, total_beats + 1e-9, beats_per_bar)
    for bt in bar_ticks:
        ax3.axvline(bt, linewidth=0.5, alpha=0.25)

    ax3.set_xlim(0, total_beats)
    ax3.set_ylim(-0.5, len(tracks)-0.5 + row_height)
    ax3.set_yticks([])
    label_every = 4
    xt = bar_ticks[::label_every]
    ax3.set_xticks(xt)
    ax3.set_xticklabels([str(int(b/beats_per_bar)+1) for b in xt])
    ax3.set_xlabel(f"Bar number (every {label_every} bars)")
    ax3.set_title("Partiture (one row per voice)")
    return fig

def player_with_red_line(midi_path, mapped_df, bars=64, time_signature=(4,4),
                         top_tracks=("Violin (P343)", "Cello (P501)"),
                         second_tracks=("Clarinet (P33)", "Trombone (P37)"),
                         width_px=1400):
    # 1) MIDI -> WAV via fluidsynth CLI
    wav_bytes = midi_to_wav_bytes_via_cli(midi_path, sr=44100)
    wav_b64 = base64.b64encode(wav_bytes).decode("ascii")

    # 2) Build combined figure -> PNG base64
    fig = build_combined_figure(mapped_df, bars=bars, time_signature=time_signature,
                                top_tracks=top_tracks, second_tracks=second_tracks)
    img_b64 = _fig_to_base64_png(fig, dpi=160)

    # 3) HTML player + synced playhead
    html = f"""
    <div style="width:{width_px}px; position:relative; font-family:sans-serif;">
      <audio id="audio" controls style="width:{width_px}px;">
        <source src="data:audio/wav;base64,{wav_b64}" type="audio/wav">
      </audio>
      <div style="width:{width_px}px; position:relative; margin-top:8px;">
        <img id="scoreimg" src="data:image/png;base64,{img_b64}" style="width:{width_px}px; display:block;"/>
        <div id="playhead" style="position:absolute; top:0; left:0; width:2px; height:100%;
             background:red; opacity:0.9; pointer-events:none;"></div>
      </div>
    </div>
    <script>
      const audio = document.getElementById('audio');
      const img = document.getElementById('scoreimg');
      const head = document.getElementById('playhead');

      function tick(){{
        const dur = audio.duration || 1;
        const t = audio.currentTime || 0;
        const frac = Math.min(1, Math.max(0, t / dur));
        const w = img.getBoundingClientRect().width;
        head.style.left = (frac * w) + "px";
        requestAnimationFrame(tick);
      }}
      img.onload = ()=>requestAnimationFrame(tick);
    </script>
    """
    display(HTML(html))

# Usage:
# player_with_red_line(midi_path, mapped, bars=64)

In [ ]:
player_with_red_line(midi_path, mapped, bars=64)